In [ ]:
# Create a BigQuery dataset
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

dataset_id = "{}.mh_dataset".format(client.project)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# TODO(developer): Specify the geographic location where the dataset should reside.
dataset.location = "EU"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))


In [ ]:
# Create a table
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = "playpen-2940ab.mh_dataset.sales"

schema = [
    bigquery.SchemaField("SalesOrderNumber", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("OrderDate", "DATE", mode="NULLABLE"),
    bigquery.SchemaField("ProductKey", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("ResellerKey", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("EmployeeKey", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("SalesTerritoryKey", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("Quantity", "INTEGER", mode="NULLABLE"),
    bigquery.SchemaField("UnitPrice", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Sales", "FLOAT", mode="NULLABLE"),
    bigquery.SchemaField("Cost", "FLOAT", mode="NULLABLE"),
]

table = bigquery.Table(table_id, schema=schema)
table = client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [ ]:
# Add a file to a bucket
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        f"File {source_file_name} uploaded to {destination_blob_name}."
    )

upload_blob("bkt-ms_dataflow", "/home/jupyter/sales.csv", "sales.csv")


In [ ]:
# Load table from CSV in bucket
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

table_id = "playpen-2940ab.mh_dataset.sales"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("SalesOrderNumber", "STRING"),
        bigquery.SchemaField("OrderDate", "DATE"),
        bigquery.SchemaField("ProductKey", "INTEGER"),
        bigquery.SchemaField("ResellerKey", "INTEGER"),
        bigquery.SchemaField("EmployeeKey", "INTEGER"),
        bigquery.SchemaField("SalesTerritoryKey", "INTEGER"),
        bigquery.SchemaField("Quantity", "INTEGER"),
        bigquery.SchemaField("UnitPrice", "FLOAT"),
        bigquery.SchemaField("Sales", "FLOAT"),
        bigquery.SchemaField("Cost", "FLOAT"),
    ],
    skip_leading_rows=1,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://bkt-ms_dataflow/sales.csv"

load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = client.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

In [ ]:
# Query a table to a dataframe
from google.cloud import bigquery
client = bigquery.Client()

sql = """
    SELECT *
    FROM `playpen-2940ab.de_dataset.Sales`
    WHERE OrderDate BETWEEN '2019-01-01' AND '2019-12-31'
"""

df = client.query_and_wait(sql).to_dataframe()

print(df)

In [ ]:
# Write dataframe to CSV
df.to_csv("2019_sales.csv")

In [33]:
# Import llm and assign it to generation_model
from vertexai.language_models import TextGenerationModel # Import a model library from the Vertex AI SDK
generation_model = TextGenerationModel.from_pretrained("text-bison@001") # Configure your model version here, the model type needs to match the imported model

In [34]:
# Pass a prompt to llm asking it to explain the SQL query run previously
prompt = "Explain this SQL statement " + QUERY # You use multi-line strings to format the prompt to be easily readable.

print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

This SQL statement selects the SalesOrderNumber, OrderDate, and ProductKey columns from the Sales table in the de_dataset dataset. The WHERE clause filters the results to only include rows where the SalesOrderNumber column is not empty and the OrderDate column is between 2020-01-01 and 2020-12-31.


In [35]:
# Add a widget that includes text box for a question and an ask button that calls the llm with the question and displays the result
import ipywidgets as widgets

# The following sets up input objects, these refer to the fields and buttons in the interface.
input1 = widgets.Text(placeholder="Ask me a question")
button = widgets.Button(description="Ask")
output = widgets.Output()

# This function defines what happens when the button is clicked.
def on_button_clicked(b):
    with output:
        question = input1.value # Set the values of location and month from the inputs
        prompt = question # Using a template string to send the model a dynamic prompt from the fields.
        print(generation_model.predict(prompt=prompt, max_output_tokens=256).text)

# This attaches the button click event to the above function.        
button.on_click(on_button_clicked)

# Display widgets and title of the app.
print("Ask me anything")
display(input1,button,output)

Ask me anything


Text(value='', placeholder='Ask me a question')

Button(description='Ask', style=ButtonStyle())

Output()